In [1]:
import numpy as np
x = np.random.rand(10,1,28,28)
x.shape

(10, 1, 28, 28)

In [2]:
print(x[0].shape)
print(x[1].shape)
print(x[0,0])

(1, 28, 28)
(1, 28, 28)
[[4.77649313e-01 6.49596899e-01 7.86248538e-01 3.34738116e-01
  2.74946304e-01 7.03231707e-01 1.92261985e-02 8.71275768e-01
  6.25780295e-01 2.57865645e-01 1.23376713e-01 3.03091397e-01
  6.14501496e-01 8.73781064e-01 1.66418221e-01 3.04526280e-01
  8.93822779e-01 7.27073001e-01 5.79930902e-01 3.97011989e-02
  1.21958442e-02 2.38033626e-02 6.17057308e-01 7.10441212e-01
  9.09025983e-01 9.86604831e-01 6.88702191e-01 6.23731678e-01]
 [7.28754761e-01 5.66524925e-01 4.05364203e-01 7.69499315e-01
  5.07053195e-01 8.96662826e-01 9.13256384e-01 3.78500740e-01
  6.16292892e-01 7.40164677e-01 8.71140570e-01 9.60727610e-01
  6.08008231e-01 8.14176304e-02 1.00049575e-01 2.80297745e-02
  6.39660430e-01 3.23428273e-01 6.72424786e-01 7.10022884e-01
  5.06524310e-01 9.12694088e-01 9.43426232e-01 9.21562353e-01
  7.59979028e-01 7.11051241e-01 6.23814628e-01 4.21450563e-01]
 [3.59117564e-01 6.26292078e-01 1.97628484e-01 2.65046982e-01
  4.74030253e-01 8.06690829e-01 1.43183193e-

# im2col method

입력 받은 이미지를 2차원 배열로 변환

In [3]:
def im2col(input_data, filter_h, filter_w, stride=1,pad=0):
    """
    파라미터
    input_data : 4차원 배열(이미지수, 채널수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    """
    
    N,C,H,W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride+1
    out_w = (H + 2*pad - filter_w)//stride+1

    img = np.pad(input_data, [(0,0),(0,0),(pad,pad),(pad,pad)],'constant')
    col = np.zeros(N,C,filter_h, filter_w, out_h, out_w)
    
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:,:,y,x,:,:] = img[:, :, y:y_max:stride, x:x_max:stride]
            
        col = col.transpose(0,4,5,1,2,3).reshape(N*out_h * out_w, -1)
        return col

# Conv2D layer

In [4]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self,x):
        FN, C, FH, FW = self.W.shape
        """
        FN : 필터의 개수
        C : 필터의 깊이
        FH : 필터의 높이
        FW : 필터의 너비
        """
        
        N, C, H, W = x.shape
        """
        N : 입력 데이터의 개수
        C : 입력 데이터의 깊이
        H : 입력 데이터의 높이
        W : 입력 데이터의 너비
        """
        
        
        out_h = (H + 2*self.pad - FH)//stride+1
        out_w = (H + 2*self.pad - FW)//stride+1
        
        col = im2col(x, FW, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1)
        
        out = np.dot(col,col_W) + self.b
        
        out = out.reshape(N, out_h,out_w,-1).transpose(0,3,1,2)
        
        return out